# Desafio Lighthouse



https://docs.google.com/document/d/1uqSPghyJWCA52ryilZrF63Ep5OzB1Yl2pBnUyaFHpS8/edit?tab=t.0#heading=h.6qxxdaq5ec9c

In [1]:
# Importando as bibliotecas necessárias
import pandas as pd
import numpy as np
import pandas_datareader.data as web
import plotly.express as px

import warnings
warnings.filterwarnings("ignore")

## Obtendo os dados na origem

In [2]:
# Declarando a url de origem
url = "https://drive.google.com/uc?export=download&id=1JUTYnugqoG_Dek-yLedjbpIHY1iJh7Ex"

# Lendo o arquivo CSV da url
df = pd.read_csv(url)

In [3]:
# Realizando uma cópia do dataframe
df_copiado = df.copy()

In [4]:
# Visualizando o início do dataframe
df_copiado.head()

,Unnamed: 0,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,1,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
1,2,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
2,3,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
3,4,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"
4,5,The Lord of the Rings: The Return of the King,2003,U,201 min,"Action, Adventure, Drama",8.9,Gandalf and Aragorn lead the World of Men agai...,94.0,Peter Jackson,Elijah Wood,Viggo Mortensen,Ian McKellen,Orlando Bloom,1642758,"377,845,905"


## Obtendo dados do FED

In [5]:
# Obtendo dados do PIB (Gross Domestic Product) dos Estados Unidos
gdp = web.DataReader("GDP", "fred", "1800-01-01")

# obtendo o PIB de anual
gdp['gdp_anual'] = gdp['GDP'].pct_change(periods=4) * 100

# Obtendo os os dados anuais do PIB (pega o último dado de cada trimestre)
gdp_anual = gdp.resample("A").last()

# Eliminando os NaNs
gdp_anual.dropna(inplace=True)

# Obtendo apenas o ano
gdp_anual.index = gdp_anual.index.year

# Ontendo dados do CPI (Consumer Price Index) dos Estados Unidos
cpi = web.DataReader("CPIAUCSL", "fred", "1800-01-01")

# Obtendo a variação anual da Infalçao dos Estados Unidos
cpi['inflacao_anual'] = cpi['CPIAUCSL'].pct_change(periods=12) * 100

# obtendo os dados anuais do CPI (pega o último dado de dezembro)
cpi_anual = cpi.resample("A").last()

# Eliminando os NaNs
cpi_anual.dropna(inplace=True)

# Obtendo apenas o ano
cpi_anual.index = cpi_anual.index.year

# visualizando os cabecalhos dos dataframes
print('Gross Domestic Product - Estados Unidos')
print(gdp_anual.head())
print('\n')
print('Consumer Price Index - Estados Unidos')
print(cpi_anual.head())

Gross Domestic Product - Estados Unidos
          GDP  gdp_anual
DATE                    
1948  280.366   7.938940
1949  270.627  -3.473674
1950  319.945  18.223607
1951  356.178  11.324759
1952  380.812   6.916205


Consumer Price Index - Estados Unidos
      CPIAUCSL  inflacao_anual
DATE                          
1948     24.05        2.733874
1949     23.61       -1.829522
1950     24.98        5.802626
1951     26.47        5.964772
1952     26.71        0.906687


## Análise, limpeza e tratamento dos dados



In [6]:
# Obtendo informações do dataframe
df_copiado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     999 non-null    int64  
 1   Series_Title   999 non-null    object 
 2   Released_Year  999 non-null    object 
 3   Certificate    898 non-null    object 
 4   Runtime        999 non-null    object 
 5   Genre          999 non-null    object 
 6   IMDB_Rating    999 non-null    float64
 7   Overview       999 non-null    object 
 8   Meta_score     842 non-null    float64
 9   Director       999 non-null    object 
 10  Star1          999 non-null    object 
 11  Star2          999 non-null    object 
 12  Star3          999 non-null    object 
 13  Star4          999 non-null    object 
 14  No_of_Votes    999 non-null    int64  
 15  Gross          830 non-null    object 
dtypes: float64(2), int64(2), object(12)
memory usage: 125.0+ KB


In [7]:
# Eliminando a coluna desnecessária
df_copiado.drop('Unnamed: 0', axis=1, inplace=True)

O DataFrame contém 999 linhas e 16 colunas. Observa-se que as colunas 'Released_Year' (ano de lançamento) e 'Gross' (faturamento) estão no formato de string e precisam ser convertidas, respectivamente, para os tipos datetime e float. Além disso, foi identificado que há dados ausentes nas colunas 'Certificate' (classificação etária), 'Meta_score' (média ponderada das críticas) e 'Gross' (faturamento).

In [8]:
# Convertendo a coluna "Released_Year" em datetime
df_copiado["Released_Year"] = pd.to_datetime(df_copiado["Released_Year"], format="%Y", errors="coerce")

In [9]:
# Convertendo a coluna  "Gross" do tipo float
df_copiado["Gross"] = df_copiado["Gross"].str.replace(",", "").astype(float)

In [10]:
# Verificando o dataframe após a conversão
df_copiado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Series_Title   999 non-null    object        
 1   Released_Year  998 non-null    datetime64[ns]
 2   Certificate    898 non-null    object        
 3   Runtime        999 non-null    object        
 4   Genre          999 non-null    object        
 5   IMDB_Rating    999 non-null    float64       
 6   Overview       999 non-null    object        
 7   Meta_score     842 non-null    float64       
 8   Director       999 non-null    object        
 9   Star1          999 non-null    object        
 10  Star2          999 non-null    object        
 11  Star3          999 non-null    object        
 12  Star4          999 non-null    object        
 13  No_of_Votes    999 non-null    int64         
 14  Gross          830 non-null    float64       
dtypes: datetime64[ns](1), f

Percebe-se que a conversão da coluna "Released_Year" não foi bem sucessida. Precisa-se averiguar com maiores detalhes

In [11]:
# Verificando a linha NaN na coluna "Released_Year"
df_copiado[df_copiado["Released_Year"].isna()]

,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
965,Apollo 13,NaT,U,140 min,"Adventure, Drama, History",7.6,NASA must devise a strategy to return Apollo 1...,77.0,Ron Howard,Tom Hanks,Bill Paxton,Kevin Bacon,Gary Sinise,269197,173837933.0


Foi realizada uma pesquisa na internet para identificar o ano de lançamento do filme Apollo 13. Constatou-se que o filme foi lançado em 1970; portanto, o dado faltante foi substituído por esse valor.

In [12]:
# Preenchendo o ano vazio
df_copiado.loc[df_copiado["Series_Title"] == "Apollo 13", "Released_Year"] = pd.to_datetime("1970", format="%Y")

# Convertendo a coluna em ano
df_copiado["Released_Year"] = df_copiado["Released_Year"].dt.year

Precisa-se averiguar com mais detalhes as colunas que contém dados faltantes.

In [13]:
# Verificando a quantidade de dados faltantes em cada coluna
df_copiado.isna().sum()

,0
Series_Title,0
Released_Year,0
Certificate,101
Runtime,0
Genre,0
IMDB_Rating,0
Overview,0
Meta_score,157
Director,0
Star1,0


Percebe-se que que a coluna "Gross" possui mais dados faltantes, em seguida vem as colunas "Meta_score" e "Certificate".

Entende-se que a remoção dos dados faltantes não será viável porque acarretaria perda de informações. Dessa forma, torna-se necessária uma análise criteriosa para definir a melhor forma de preenchê-los.

Iniciaremos a investigação pela coluna 'Gross'.

In [14]:
# Visualizando as linhas que contém NaNs na coluna Gross
df_copiado[df_copiado['Gross'].isna()]

,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
17,Hamilton,2020,PG-13,160 min,"Biography, Drama, History",8.6,The real life of one of America's foremost fou...,90.0,Thomas Kail,Lin-Manuel Miranda,Phillipa Soo,Leslie Odom Jr.,Renée Elise Goldsberry,55291,NaN
19,Soorarai Pottru,2020,U,153 min,Drama,8.6,"Nedumaaran Rajangam ""Maara"" sets out to make t...",NaN,Sudha Kongara,Suriya,Madhavan,Paresh Rawal,Aparna Balamurali,54995,NaN
29,Seppuku,1962,NaN,133 min,"Action, Drama, Mystery",8.6,When a ronin requesting seppuku at a feudal lo...,85.0,Masaki Kobayashi,Tatsuya Nakadai,Akira Ishihama,Shima Iwashita,Tetsurô Tanba,42004,NaN
31,It's a Wonderful Life,1946,PG,130 min,"Drama, Family, Fantasy",8.6,An angel is sent from Heaven to help a despera...,89.0,Frank Capra,James Stewart,Donna Reed,Lionel Barrymore,Thomas Mitchell,405801,NaN
45,Hotaru no haka,1988,U,89 min,"Animation, Drama, War",8.5,A young boy and his little sister struggle to ...,94.0,Isao Takahata,Tsutomu Tatsumi,Ayano Shiraishi,Akemi Yamaguchi,Yoshiko Shinohara,235231,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
992,Blowup,1966,A,111 min,"Drama, Mystery, Thriller",7.6,A fashion photographer unknowingly captures a ...,82.0,Michelangelo Antonioni,David Hemmings,Vanessa Redgrave,Sarah Miles,John Castle,56513,NaN
994,Breakfast at Tiffany's,1961,A,115 min,"Comedy, Drama, Romance",7.6,A young New York socialite becomes interested ...,76.0,Blake Edwards,Audrey Hepburn,George Peppard,Patricia Neal,Buddy Ebsen,166544,NaN
995,Giant,1956,G,201 min,"Drama, Western",7.6,Sprawling epic covering the life of a Texas ca...,84.0,George Stevens,Elizabeth Taylor,Rock Hudson,James Dean,Carroll Baker,34075,NaN
997,Lifeboat,1944,NaN,97 min,"Drama, War",7.6,Several survivors of a torpedoed merchant ship...,78.0,Alfred Hitchcock,Tallulah Bankhead,John Hodiak,Walter Slezak,William Bendix,26471,NaN


Observa-se que os filmes sem dados de faturamento possuem, muitas vezes, notas relevantes no IMDB, quantidade de votos e na média ponderada das críticas. Calcular a média de faturamento por gênero não é adequado, pois o faturamento varia ao longo do tempo e depende do período de lançamento dos filmes.

A saída esperada consiste no preenchimento das lacunas na coluna de faturamento "Gross" utilizando a média por período de lançamento.

In [15]:
# Calculando a média de Gross por ano
media_por_ano = df_copiado.groupby('Released_Year')['Gross'].mean()

# Visualizando as médias obtidas por ano
print(media_por_ano)

Released_Year
1920             NaN
1921    5.450000e+06
1922             NaN
1924    9.773750e+05
1925    2.750485e+06
            ...     
2016    1.128503e+08
2017    1.030656e+08
2018    1.862684e+08
2019    1.504214e+08
2020             NaN
Name: Gross, Length: 99, dtype: float64


Observa-se que existem períodos sem informações suficientes para preencher a média de faturamento. A seguir, vamos identificar quais anos não possuem esses dados.

In [16]:
# visualizando os anos que não possem dados médios de faturamento
media_por_ano[media_por_ano.isna()]

,Gross
Released_Year,
1920,NaN
1922,NaN
1932,NaN
1943,NaN
2020,NaN


In [17]:
# preenchendo os dados com a média obtida em cada período
df_copiado['Gross'] = df_copiado.apply(
    lambda row: media_por_ano[row['Released_Year']] if pd.isna(row['Gross']) else row['Gross'],
    axis=1
)

In [18]:
# Verificando quantidade de linhas que ainda possuem os NaNs
df_copiado['Gross'].isna().sum()

np.int64(11)

Diante dessa situação, os dados faltantes na coluna "Gross"" serão preenchidos por zero

In [19]:
# preeenchendo as lacunas por zero
df_copiado['Gross'].fillna(0, inplace=True)

In [20]:
# Verificando se ficou com dados ausentes
df_copiado['Gross'].isna().sum()

np.int64(0)

Vamos partir para a próxima coluna "Meta_score"

In [21]:
# Visualizando as linhas que contém NaNs na coluna Meta_score
df_copiado[df_copiado['Meta_score'].isna()]

,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
19,Soorarai Pottru,2020,U,153 min,Drama,8.6,"Nedumaaran Rajangam ""Maara"" sets out to make t...",NaN,Sudha Kongara,Suriya,Madhavan,Paresh Rawal,Aparna Balamurali,54995,0.000000e+00
53,Ayla: The Daughter of War,2017,NaN,125 min,"Biography, Drama, History",8.4,"In 1950, amid-st the ravages of the Korean War...",NaN,Can Ulkay,Erdem Can,Çetin Tekindor,Ismail Hacioglu,Kyung-jin Lee,34112,1.030656e+08
54,Vikram Vedha,2017,UA,147 min,"Action, Crime, Drama",8.4,"Vikram, a no-nonsense police officer, accompan...",NaN,Gayatri,Pushkar,Madhavan,Vijay Sethupathi,Shraddha Srinath,28401,1.030656e+08
56,Dangal,2016,U,161 min,"Action, Biography, Drama",8.4,Former wrestler Mahavir Singh Phogat and his t...,NaN,Nitesh Tiwari,Aamir Khan,Sakshi Tanwar,Fatima Sana Shaikh,Sanya Malhotra,156479,1.239176e+07
64,Taare Zameen Par,2007,U,165 min,"Drama, Family",8.4,An eight-year-old boy is thought to be a lazy ...,NaN,Aamir Khan,Amole Gupte,Darsheel Safary,Aamir Khan,Tisca Chopra,168895,1.223869e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
871,The Day the Earth Stood Still,1951,U,92 min,"Drama, Sci-Fi",7.7,An alien lands and tells the people of Earth t...,NaN,Robert Wise,Michael Rennie,Patricia Neal,Hugh Marlowe,Sam Jaffe,76315,5.034003e+06
873,Gilda,1946,Approved,110 min,"Drama, Film-Noir, Romance",7.7,A small-time gambler hired to work in a Buenos...,NaN,Charles Vidor,Rita Hayworth,Glenn Ford,George Macready,Joseph Calleia,27991,1.355133e+07
898,El cuerpo,2012,NaN,112 min,"Mystery, Thriller",7.6,A detective searches for the body of a femme f...,NaN,Oriol Paulo,Jose Coronado,Hugo Silva,Belén Rueda,Aura Garrido,57549,1.210770e+08
908,Celda 211,2009,NaN,113 min,"Action, Adventure, Crime",7.6,The story of two men on different sides of a p...,NaN,Daniel Monzón,Luis Tosar,Alberto Ammann,Antonio Resines,Manuel Morón,63882,1.174868e+08


Vamos aplicar o mesmo raciocínio utilizado anteriormente na coluna 'Gloss' à coluna 'Meta_score'.

In [22]:
# Calculando a média de Gross por ano
media_por_ano = df_copiado.groupby('Released_Year')['Meta_score'].mean()

# Visualizando as médias obtidas por ano
print(media_por_ano)

Released_Year
1920          NaN
1921          NaN
1922          NaN
1924          NaN
1925    97.000000
          ...    
2016    78.904762
2017    79.473684
2018    77.692308
2019    76.904762
2020    82.750000
Name: Meta_score, Length: 99, dtype: float64


A partir da análise anterior, observa-se que nos anos iniciais não existe uma média ponderada de todas as críticas. Nesses casos, os valores serão substituídos por zero, enquanto para os demais anos utilizaremos a média.

In [23]:
# preenchendo os dados com a média obtida em cada período
df_copiado['Meta_score'] = df_copiado.apply(
    lambda row: media_por_ano[row['Released_Year']] if pd.isna(row['Meta_score']) else row['Meta_score'],
    axis=1
)

In [24]:
# preeenchendo as lacunas por zero
df_copiado['Meta_score'].fillna(0, inplace=True)

In [25]:
# Verificando quantidade de linhas que ainda possuem os NaNs
df_copiado['Meta_score'].isna().sum()

np.int64(0)

Por último, vamos analisar a coluna "Certificate"

In [26]:
# Filtrando a coluna "Certificate"
df_copiado[df_copiado['Certificate'].isna()]

,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
29,Seppuku,1962,NaN,133 min,"Action, Drama, Mystery",8.6,When a ronin requesting seppuku at a feudal lo...,85.000000,Masaki Kobayashi,Tatsuya Nakadai,Akira Ishihama,Shima Iwashita,Tetsurô Tanba,42004,2.932471e+07
53,Ayla: The Daughter of War,2017,NaN,125 min,"Biography, Drama, History",8.4,"In 1950, amid-st the ravages of the Korean War...",79.473684,Can Ulkay,Erdem Can,Çetin Tekindor,Ismail Hacioglu,Kyung-jin Lee,34112,1.030656e+08
76,Tengoku to jigoku,1963,NaN,143 min,"Crime, Drama, Mystery",8.4,An executive of a shoe company becomes a victi...,87.500000,Akira Kurosawa,Toshirô Mifune,Yutaka Sada,Tatsuya Nakadai,Kyôko Kagawa,34357,9.257202e+06
91,Babam ve Oglum,2005,NaN,112 min,"Drama, Family",8.3,The family of a left-wing journalist is torn a...,71.666667,Çagan Irmak,Çetin Tekindor,Fikret Kuskan,Hümeyra,Ege Tanman,78925,6.825130e+07
120,Ikiru,1952,NaN,143 min,Drama,8.3,A bureaucrat tries to find a meaning in his li...,94.000000,Akira Kurosawa,Takashi Shimura,Nobuo Kaneko,Shin'ichi Himori,Haruo Tanaka,68463,5.524000e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
919,The Secret of Kells,2009,NaN,71 min,"Animation, Adventure, Family",7.6,A young boy in a remote medieval outpost under...,81.000000,Tomm Moore,Nora Twomey,Evan McGuire,Brendan Gleeson,Mick Lally,31779,6.863830e+05
925,Dead Man's Shoes,2004,NaN,90 min,"Crime, Drama, Thriller",7.6,A disaffected soldier returns to his hometown ...,52.000000,Shane Meadows,Paddy Considine,Gary Stretch,Toby Kebbell,Stuart Wolfenden,49728,6.013000e+03
943,Batoru rowaiaru,2000,NaN,114 min,"Action, Adventure, Drama",7.6,"In the future, the Japanese government capture...",81.000000,Kinji Fukasaku,Tatsuya Fujiwara,Aki Maeda,Tarô Yamamoto,Takeshi Kitano,169091,5.333543e+07
997,Lifeboat,1944,NaN,97 min,"Drama, War",7.6,Several survivors of a torpedoed merchant ship...,78.000000,Alfred Hitchcock,Tallulah Bankhead,John Hodiak,Walter Slezak,William Bendix,26471,5.040000e+06


Para esta variável, vamos utilizar a coluna "Genre" para identificar e realizar o preenchimento

In [27]:
# Criando um mapa
mapa = (
    df_copiado
    .dropna(subset=["Certificate"])
    .groupby("Genre")["Certificate"]
    .agg(lambda x: x.mode().iat[0])
    .to_dict()
)

In [28]:
# Preenchendo os NaNs de acordo com o Gênero
df_copiado["Certificate"] = df_copiado.apply(
    lambda row: mapa.get(row["Genre"], row["Certificate"]) if pd.isna(row["Certificate"]) else row["Certificate"],
    axis=1
)

In [29]:
# Verificando se há dados ausentes
df_copiado['Certificate'].isna().sum()

np.int64(11)

Constata-se que ainda existem 11 filmes sem classificação.

In [30]:
# Visualizando os 11 filmes sem classificação
df_copiado[df_copiado['Certificate'].isna()]

,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
291,El ángel exterminador,1962,NaN,95 min,"Drama, Fantasy",8.1,The guests at an upper-class dinner party find...,87.111111,Luis Buñuel,Silvia Pinal,Jacqueline Andere,Enrique Rambal,José Baviera,29682,2.932471e+07
320,Das Cabinet des Dr. Caligari,1920,NaN,76 min,"Fantasy, Horror, Mystery",8.1,"Hypnotist Dr. Caligari uses a somnambulist, Ce...",0.000000,Robert Wiene,Werner Krauss,Conrad Veidt,Friedrich Feher,Lil Dagover,57428,0.000000e+00
389,Knockin' on Heaven's Door,1997,NaN,87 min,"Action, Crime, Comedy",8.0,Two terminally ill patients escape from a hosp...,69.500000,Thomas Jahn,Til Schweiger,Jan Josef Liefers,Thierry van Werveke,Moritz Bleibtreu,27721,3.296000e+03
443,Les diaboliques,1955,NaN,117 min,"Crime, Drama, Horror",8.0,The wife and mistress of a loathed school prin...,89.600000,Henri-Georges Clouzot,Simone Signoret,Véra Clouzot,Paul Meurisse,Charles Vanel,61503,3.556130e+05
454,Arsenic and Old Lace,1942,NaN,118 min,"Comedy, Crime, Thriller",8.0,A writer of books on the futility of marriage ...,93.000000,Frank Capra,Cary Grant,Priscilla Lane,Raymond Massey,Jack Carson,65101,1.024560e+06
455,The Maltese Falcon,1941,NaN,100 min,"Film-Noir, Mystery",8.0,A private detective takes on a case that invol...,96.000000,John Huston,Humphrey Bogart,Mary Astor,Gladys George,Peter Lorre,148928,2.108060e+06
539,"Aguirre, der Zorn Gottes",1972,NaN,95 min,"Action, Adventure, Biography",7.9,"In the 16th century, the ruthless and insane D...",88.600000,Werner Herzog,Klaus Kinski,Ruy Guerra,Helena Rojo,Del Negro,52397,3.180164e+07
567,Nosferatu,1922,NaN,94 min,"Fantasy, Horror",7.9,Vampire Count Orlok expresses interest in a ne...,0.000000,F.W. Murnau,Max Schreck,Alexander Granach,Gustav von Wangenheim,Greta Schröder,88794,0.000000e+00
699,Wait Until Dark,1967,NaN,108 min,Thriller,7.8,A recently blinded woman is terrorized by a tr...,81.000000,Terence Young,Audrey Hepburn,Alan Arkin,Richard Crenna,Efrem Zimbalist Jr.,27733,1.755074e+07
716,Duck Soup,1933,NaN,69 min,"Comedy, Musical, War",7.8,Rufus T. Firefly is named president/dictator o...,93.000000,Leo McCarey,Groucho Marx,Harpo Marx,Chico Marx,Zeppo Marx,55581,1.000000e+07


Recomenda-se utilizar a primeira categoria do campo de gênero (a primeira categoria da coluna Genre) como critério da classificação.

In [31]:
# Preenchendo
df_copiado.loc[df_copiado["Certificate"].isna(), "Certificate"] = (
    df_copiado.loc[df_copiado["Certificate"].isna(), "Genre"]
    .str.split(",")
    .str[0]
    .str.strip()
)

In [32]:
# Verificando se há dados ausentes
df_copiado['Certificate'].isna().sum()

np.int64(0)

In [33]:
# Agora vamos visualizar se há algum dado ausente no dataframe
df_copiado.isna().sum()

,0
Series_Title,0
Released_Year,0
Certificate,0
Runtime,0
Genre,0
IMDB_Rating,0
Overview,0
Meta_score,0
Director,0
Star1,0


## Análise Exploratória de dados

Iniciaremos a análise observando a evolução do faturamento ao longo dos anos, tomando como referência o ano de lançamento dos filmes.

In [34]:
# Agrupando os dados por ano e somando o faturamento
df_faturamento = df_copiado.groupby("Released_Year", as_index=False)["Gross"].sum()

# Criando a figura do gráfico de barras
fig = px.bar(
    df_faturamento,
    x="Released_Year",
    y="Gross",
    title="Evolução do Faturamento por Ano de Lançamento",
    labels={"Released_Year": "Ano de Lançamento", "Gross": "Faturamento (US$)"},
)

# Atualizando layout
fig.update_layout(
    width=950,
    height=450,
    template="plotly_white",
    title_font_size=20,
    xaxis_tickangle=-45,
    yaxis_tickprefix="$",
    yaxis_tickformat=",",  # Adiciona separador de milhar
)

# Exibindo o gráfico
fig.show()

O gráfico evidencia que nos últimos anos, houve um aumento expressivo no faturamento, porem em determinados períodos, o faturamento apresentou queda, possivelmente pode está associado à eventos externos como por exemplo, o aumento da inflação nos Estados Unidos. Para investigar essa relação, será realizada uma análise de correlação considerando os dados do PIB e da inflação norte-americana.

In [35]:
# Concatenando os dataframes utilizando o método 'inner', pois pretende-se obter apenas os dados que estão contidos no dataframe do IMDB
df_concatenado = (
    df_copiado
    .merge(gdp_anual, left_on="Released_Year", right_index=True, how="inner")
    .merge(cpi_anual, left_on="Released_Year", right_index=True, how="inner")
)

# Verificando a correlação entre as variáveis numéricas
df_concatenado.corr(numeric_only=True)

,Released_Year,IMDB_Rating,Meta_score,No_of_Votes,Gross,GDP,gdp_anual,CPIAUCSL,inflacao_anual
Released_Year,1.000000,-0.115700,-0.295667,0.205644,0.239983,0.953349,-0.482123,0.987194,-0.294948
IMDB_Rating,-0.115700,1.000000,0.263726,0.495986,0.098624,-0.093836,0.057128,-0.108606,0.012001
Meta_score,-0.295667,0.263726,1.000000,-0.011952,-0.039093,-0.206274,0.118587,-0.267865,0.024870
No_of_Votes,0.205644,0.495986,-0.011952,1.000000,0.561575,0.159837,-0.100269,0.196200,-0.049859
Gross,0.239983,0.098624,-0.039093,0.561575,1.000000,0.240305,-0.127308,0.234259,-0.045151
GDP,0.953349,-0.093836,-0.206274,0.159837,0.240305,1.000000,-0.524760,0.977588,-0.397094
gdp_anual,-0.482123,0.057128,0.118587,-0.100269,-0.127308,-0.524760,1.000000,-0.533719,0.611092
CPIAUCSL,0.987194,-0.108606,-0.267865,0.196200,0.234259,0.977588,-0.533719,1.000000,-0.369317
inflacao_anual,-0.294948,0.012001,0.024870,-0.049859,-0.045151,-0.397094,0.611092,-0.369317,1.000000


A partir da análise da correlação tem-se:

- A coluna "Gross" possui correlação moderada e positiva com a coluna "No_of_Votes", o que sugere que a quantidade de votos de usuários está associada positivamente ao faturamento.
- A coluna "IMDB_Rating" possui correlação moderada e positiva com a coluna "No_of_Votes", indicando que a classificação no IMDB está associada positivamente à quantidade de votos de usuários.
- A coluna "Meta_score" possui correlação fraca a moderada negativa com a coluna "Released_Year", sugerindo que a média ponderada das críticas está associada negativamente com o ano de lançamento.
- As colunas "No_of_Votes" e "Gross" possuem correlação fraca com a coluna "Released_Year", indicando que a quantidade de votos e o faturamento não sofrem grandes efeitos em função do ano de lançamento.
- As colunas "GDP" e "CPIAUCSL" possuem correlação fraca e positiva com a coluna "Gross", sugerindo que o faturamento dos filmes não sofre efeitos severos em relação ao PIB e à inflação dos Estados Unidos.
- As colunas "gdp_anual" e "inflacao_anual" apresentam correlação fraca a negativa com a coluna "Gross", sugerindo que as variações anuais do PIB e da inflação norte-americana não exercem efeitos relevantes sobre o faturamento dos filmes.

Em seguida, é realizada uma análise descritiva nas variáveis numéricas

In [36]:
# Aplicando o describe para visualizar (média, descio padrão, mínimo, máximo, o primeiro quartil, segundo quartil, terceiro quartil)
df_copiado.describe().round(2)

,Released_Year,IMDB_Rating,Meta_score,No_of_Votes,Gross
count,999.00,999.00,999.00,999.00,9.990000e+02
mean,1991.19,7.95,77.98,271621.42,6.551194e+07
std,23.31,0.27,13.28,320912.62,1.022390e+08
min,1920.00,7.60,0.00,25088.00,0.000000e+00
25%,1976.00,7.70,72.00,55471.50,4.009348e+06
50%,1999.00,7.90,79.00,138356.00,2.694762e+07
75%,2009.00,8.10,87.00,373167.50,8.353862e+07
max,2020.00,9.20,100.00,2303232.00,9.366622e+08


Vamos calcular o coeficiente de variação e verificar a dispersão relativa do faturamento e checar se os dados são heterogênios em relação à média.

In [37]:
# calculando o cv
cv = (df_copiado['Gross'].std() / df_copiado['Gross'].mean()) * 100

# Imprimindo na tela
print(f"O Coeficiente de Variação é: {cv.round(2)} %")

O Coeficiente de Variação é: 156.06 %


Neste contexto, o faturamento apresenta altíssima dispersão relativa, ou seja, os dados são heterogêneos em relação à média.

Em seguida, foi plotado um gráfico de histograma para visualizar a distribuição dos dados de faturamento.

In [38]:
# Criando uma figura de histograma
fig = px.histogram(
    df_copiado,
    x="Gross",
    title="Histograma do Faturamento dos Filmes",
    labels={"Gross": "Faturamento (US$)"},
)

# Atualizando layout
fig.update_layout(
    width=950,
    height=450,
    template="plotly_white",
    title_font_size=20,
    xaxis_tickangle=-45,
    yaxis_tickprefix="",
    yaxis_tickformat=",",
)

# Exibindo o gráfico
fig.show()

Observa-se uma maior concentração de filmes com faturamento baixo, enquanto apenas poucos alcançam faturamentos muito elevados.

Outra variável que chamou atenção foi a variável "No_of_votes", na análise descritiva a amplitude é muito alta, percebe-se claramente que há outliers nesta variável.

In [39]:
# Criando uma figura de boxplot
fig = px.box(
    df_copiado,
    x="No_of_Votes",
    title="Boxplot - número de votos",
    labels={"No_of_Votes": "Número de votos"},
)

# Atualizando layout
fig.update_layout(
    width=950,
    height=450,
    template="plotly_white",
    title_font_size=20,
    xaxis_tickangle=-45,
    yaxis_tickprefix="",
    yaxis_tickformat=",",
)

# Exibindo o gráfico
fig.show()

Confirmado a presença de outliers na variável, indicando que certos filmes obtiveram um número de votos excepcionalmente alto em comparação com os demais.

Em seguida, verificamos as notas que possuem maiores faturamentos

In [40]:
# Criando a figura do gráfico de barras
fig = px.histogram(
    df_copiado,
    x="IMDB_Rating",
    y="Gross",
    title="Notas IMDB por faturamento",
    labels={"IMDB_Rating": "Nota IMDB", "Gross": "Faturamento"},
)

# Atualizando layout
fig.update_layout(
    width=950,
    height=450,
    template="plotly_white",
    title_font_size=20,
    xaxis_tickangle=-45,
    yaxis_tickprefix="$",
    yaxis_tickformat=".",
)

# Exibindo o gráfico
fig.show()

Observa-se que os filmes com notas IMBD próximo a 8 possuem maior faturamento

O gráfico a seguir, aborda os 10 filmes com maiores quantidade de votos por faturamento

In [41]:
# Ordenando do maior para o menor
nota = df_copiado.sort_values(by="No_of_Votes", ascending=False).head(10)

# Criando o gráfico de barras
fig = px.bar(
    nota,
    x="Series_Title",
    y="Gross",
    title="Filmes com maior quantidade de votos por faturamento",
    labels={"Series_Title": "Filmes", "Gross": "Faturamento"},
    text="Gross",
)

# Atualizando layout
fig.update_layout(
    width=1000,
    height=600,
    template="plotly_white",
    title_font_size=22,
    xaxis_tickangle=-45,
    yaxis_title="Faturamento ($)",
    xaxis_title="Filmes",
)

# Formatação do eixo y (milhões ou bilhões)
fig.update_yaxes(tickprefix="$", tickformat=",")  # vírgula como separador de milhares

# Ajuste do tamanho do texto nas barras
fig.update_traces(texttemplate="$%{text:,}", textposition="outside")

# Exibindo o gráfico
fig.show()

O próximo gráfico apresenta os 10 diretores que mais pulicaram filmes.

In [42]:
# Criando um dataframe para agrupar por diretor
df_diretor = df_copiado.groupby("Director", as_index=False)["Series_Title"].count()

# Ordenando do maior para o menor
df_diretor = df_diretor.sort_values(by="Series_Title", ascending=False).head(10)

# Criando a figura do gráfico de barras
fig = px.bar(
    df_diretor,
    x="Director",
    y="Series_Title",
    title="Os 10 diretores que mais publicaram filmes",
    labels={"Director": "Diretores", "Series_Title": "Quantidade de filmes"},
)

# Atualizando layout
fig.update_layout(
    width=950,
    height=450,
    template="plotly_white",
    title_font_size=20,
    xaxis_tickangle=-45,
    yaxis_tickprefix="",
    yaxis_tickformat=".",
)

# Exibindo o gráfico
fig.show()

Os 10 diretores com maior faturamento

In [43]:
# Criando um dataframe para agrupar por diretor
df_diretor_fat = df_copiado.groupby("Director", as_index=False)["Gross"].sum()

# Ordenando do maior para o menor
df_diretor_fat = df_diretor_fat.sort_values(by="Gross", ascending=False).head(10)

# Criando a figura do gráfico de barras
fig = px.bar(
    df_diretor_fat,
    x="Director",
    y="Gross",
    title="Os 10 diretores que mais faturaram",
    labels={"Director": "Diretores", "Gross": "Faturamento"},
    text="Gross"
)

# Atualizando layout
fig.update_layout(
    width=950,
    height=450,
    template="plotly_white",
    title_font_size=20,
    xaxis_tickangle=-45,
    yaxis_tickprefix="R",
    yaxis_tickformat=".",
)

# Exibindo o gráfico
fig.show()

Os 10 gêneros de filmes mais publicados até o momento.

In [44]:
# Criando um dataframe para agrupar por gênero
df_genero = df_copiado.groupby("Genre", as_index=False)["Series_Title"].count()

# Ordenando do maior para o menor
df_genero = df_genero.sort_values(by="Series_Title", ascending=False).head(10)

# Criando a figura do gráfico de barras
fig = px.bar(
    df_genero,
    x="Genre",
    y="Series_Title",
    title="Os 10 gêneros mais publicados",
    labels={"Genre": "Gênero", "Series_Title": "Quantidade de filmes"},
)

# Atualizando layout
fig.update_layout(
    width=950,
    height=450,
    template="plotly_white",
    title_font_size=20,
    xaxis_tickangle=-45,
    yaxis_tickprefix="",
    yaxis_tickformat=".",
)

# Exibindo o gráfico
fig.show()

Os filmes com maiores faturamento

In [45]:
# Ordenando o faturamento
df_ordenado_1 = df_copiado.sort_values(by="Gross", ascending=False).head(10)

# Criando a figura do gráfico de barras
fig = px.bar(
    df_ordenado_1,
    x="Gross",
    y="Series_Title",
    orientation="h",
    color="Gross",
    color_continuous_scale="blues",
    title="Os 10 filmes mais faturados",
    labels={"Gross": "Faturamento", "Series_Title": "Filme"},
)

# Atualizando layout
fig.update_layout(
    width=950,
    height=500,
    template="plotly_white",
    title_font_size=20,
    xaxis_title="Faturamento",
    yaxis_title="",
    yaxis=dict(autorange="reversed"),  # para o melhor filme ficar no topo
)

# Adicionando rótulos com as notas
fig.update_traces(texttemplate='%{x:.1f}', textposition='outside')

# Exibindo o gráfico
fig.show()

Os filmes com maiores notas IMDB

In [46]:
# Ordenando do maior para o menor
df_ordenado_2 = df_copiado.sort_values(by="IMDB_Rating", ascending=False).head(10)

# Criando a figura do gráfico de barras
fig = px.bar(
    df_ordenado_2,
    x="IMDB_Rating",
    y="Series_Title",
    orientation="h",
    color="IMDB_Rating",
    color_continuous_scale="blues",
    title="Os 10 filmes com maiores notas IMDB",
    labels={"IMDB_Rating": "Nota IMDB", "Series_Title": "Filme"},
)

# Atualizando layout
fig.update_layout(
    width=950,
    height=500,
    template="plotly_white",
    title_font_size=20,
    xaxis_title="Nota IMDB",
    yaxis_title="",
    yaxis=dict(autorange="reversed"),  # para o melhor filme ficar no topo
)

# Adicionando rótulos com as notas
fig.update_traces(texttemplate='%{x:.1f}', textposition='outside')

# Exibindo o gráfico
fig.show()

Os 10 filmes mais votados

In [47]:
# Ordenando do maior para o menor
df_ordenado_3 = df_copiado.sort_values(by="No_of_Votes", ascending=False).head(10)

# Criando a figura do gráfico de barras
fig = px.bar(
    df_ordenado_3,
    x="No_of_Votes",
    y="Series_Title",
    orientation="h",
    color="No_of_Votes",
    color_continuous_scale="blues",
    title="Os 10 filmes com maiores número de votos",
    labels={"No_of_Votes": "nº de votos", "Series_Title": "Filme"},
)

# Atualizando layout
fig.update_layout(
    width=950,
    height=500,
    template="plotly_white",
    title_font_size=20,
    xaxis_title="nº votos",
    yaxis_title="",
    yaxis=dict(autorange="reversed"),  # para o melhor filme ficar no topo
)

# Adicionando rótulos com as notas
fig.update_traces(texttemplate='%{x:.1f}', textposition='outside')

# Exibindo o gráfico
fig.show()

## Normalização dos dados para modelo de previsão (Regressão)

In [48]:
# Instalando a biblioteca
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from sklearn.model_selection import train_test_split

In [49]:
# Criando uma cópia do datarframe
df_copiado_2 = df_copiado.copy()

In [50]:
# Obtendo as colunas necessárias
df_features = df_copiado_2[['Released_Year', 'Runtime', 'Certificate', 'Genre', 'IMDB_Rating', 'Meta_score', 'No_of_Votes', 'Gross']]

In [51]:
# Dividindo a coluna "Runtime" e obtendo apenas o valor
df_features['Runtime_clean'] = df_features['Runtime'].str.replace('min', '').str.strip()

# Eliminando a coluna Runtime
df_features = df_features.drop('Runtime', axis=1)

In [52]:
# Criando uma coluna de Genre_lista e separarando os gêneros em listas
df_features['Genre_lista'] = df_features['Genre'].str.split(', ')

In [53]:
# Instanciando o One-Hot Encoding - MultiLabelBinarizer
mlb = MultiLabelBinarizer()

# Aplicando na coluna Genre_lista
genre_encoded = mlb.fit_transform(df_features['Genre_lista'])

In [54]:
# Convertendo em dataframe
df_genres = pd.DataFrame(genre_encoded, columns=mlb.classes_)

In [55]:
# Aplicando o dummies na coluna Certificate
df_cert = pd.get_dummies(df_features['Certificate'], prefix='Cert', dtype=int)

In [56]:
# Concatenando o datagrame df_genres no dataframe df_features
df_features = pd.concat([df_features, df_genres, df_cert], axis=1)

# Eliminando as colunas categóricas
df_features = df_features.drop(['Genre','Certificate','Genre_lista'], axis=1)

In [57]:
# Algumas colunas precisam ser normalizadas para que o modelo de previsão consiga prever corretamente

# Instanciando StandardScaler
scaler = StandardScaler()

# Aplicando a normalização nas colunas numéricas
df_features[['Runtime_clean', 'IMDB_Rating', 'Meta_score', 'No_of_Votes', 'Gross']] = scaler.fit_transform(
    df_features[['Runtime_clean', 'IMDB_Rating', 'Meta_score', 'No_of_Votes', 'Gross']]
)

Antes de dividir os dados em conjuntos de treino e teste, é importante verificar se há colinearidade entre as variáveis. A presença de multicolinearidade pode comprometer a performance e a interpretação dos modelos, especialmente em algoritmos lineares.

In [58]:
# Obtendo os dados das features
X = df_features.drop(columns=['IMDB_Rating'])

# Adiciona intercepto
X_const = add_constant(X)

# Calculando VIF
vif = pd.DataFrame()
vif["Feature"] = X_const.columns
vif["VIF"] = [variance_inflation_factor(X_const.values, i)
              for i in range(X_const.shape[1])]

# Imprimindo na tela
print(vif)

           Feature       VIF
0            const  0.000000
1    Released_Year  2.018700
2       Meta_score  1.275376
3      No_of_Votes  1.735470
4            Gross  1.906826
5    Runtime_clean  1.427047
6           Action  1.658962
7        Adventure  1.870476
8        Animation  1.682340
9        Biography  1.351224
10          Comedy  1.676737
11           Crime  1.434966
12           Drama  2.233373
13          Family  1.400886
14         Fantasy  1.229276
15       Film-Noir  1.549122
16         History  1.250168
17          Horror  1.333904
18           Music  1.135962
19         Musical  1.164586
20         Mystery  1.246928
21         Romance  1.325874
22          Sci-Fi  1.323395
23           Sport  1.097731
24        Thriller  1.382215
25             War  1.174932
26         Western  1.152901
27         Cert_16       inf
28          Cert_A       inf
29     Cert_Action       inf
30   Cert_Approved       inf
31     Cert_Comedy       inf
32      Cert_Crime       inf
33      Cert_D

A partir dos resultados obtidos pelo VIF (Variance Inflation Factor), observa-se que não há multicolinearidade nas variáveis relacionadas ao gênero. No entanto, as variáveis associadas aos certificados apresentaram indícios de colinearidade.

In [59]:
# Nesta etapa, será necessário a divisão de dados de treino e teste

# Obtendo as variáveis independentes e dependente
X = df_features.drop('IMDB_Rating', axis=1) # Variáveis independentes
y = df_features['IMDB_Rating'] # Variável dependente

# Separando os dados em treino e teste (70% para treino e 30% para teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## Modelos

## Modelos de regressão

Os modelos de regressão que serão utilizados são:
- Ridge Regression
- Lasso Regression
- ElasticNet
- Random Forest Regression
- Decision Tree Regression
- XGBoost Regression

Para avaliação do modelo de regressão linear será empregado o Erro Quadrático Médio (Mean Squared Error)

### Ridge Regression

In [60]:
# importando a biblioteca
from sklearn.linear_model import Ridge, Lasso, ElasticNet

# isntanciando o modelo
ridge = Ridge(alpha=1.0)

# ajuste do modelo com os dados de teste
ridge.fit(X_train, y_train)

Ridge()

In [61]:
# realizando a predição com os dados de teste
y_pred_ridge = ridge.predict(X_test)

In [62]:
# Avaliando o modelo

# importando a biblioteca
from sklearn.metrics import mean_squared_error

ridge_mse = mean_squared_error(y_test, y_pred_ridge)
print("Mean Squared Error:", np.sqrt(ridge_mse))

Mean Squared Error: 0.8060665682997251


### Lasso Regression

In [63]:
# isntanciando o modelo
lasso = Lasso(alpha=0.01)

# Treinando o modelo
lasso.fit(X_train, y_train)

# Prevendo
lasso_pred = lasso.predict(X_test)

In [64]:
lasso_mse = mean_squared_error(y_test, lasso_pred)
print("Mean Squared Error:", np.sqrt(lasso_mse))

Mean Squared Error: 0.7962726540478436


### ElasticNet

In [65]:
# isntanciando o modelo
elastic = ElasticNet(alpha=0.01, l1_ratio=0.5)

# Treinando o modelo
elastic.fit(X_train, y_train)

# Prevendo
elastic_pred = elastic.predict(X_test)

In [66]:
elastic_mse = mean_squared_error(y_test, elastic_pred)
print("Mean Squared Error:", np.sqrt(elastic_mse))

Mean Squared Error: 0.7972117925946672


### Random Forest Regression

In [67]:
# Importando a biblioteca necessária
from sklearn.ensemble import RandomForestRegressor

# Treinar o modelo Random Forest
random_forest = RandomForestRegressor(n_estimators=100, random_state=42)
random_forest.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred_forest = random_forest.predict(X_test)

In [68]:
# Avaliando o modelo
forest_mse = mean_squared_error(y_test, y_pred_forest)
print("Mean Squared Error:", np.sqrt(forest_mse))

Mean Squared Error: 0.7266667133513585


### Decision Tree Regression

In [71]:
# Importando a biblioteca necessária
from sklearn.tree import DecisionTreeRegressor

# Criar e treinar o modelo MLPRegressor
decision_tree = DecisionTreeRegressor()
decision_tree.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred_decision = decision_tree.predict(X_test)

In [72]:
# Avaliando o modelo
tree_mse = mean_squared_error(y_test, y_pred_decision)
print("Mean Squared Error:", np.sqrt(tree_mse))

Mean Squared Error: 0.9767734998323248


### XGBoost Regression

In [69]:
# Importando a biblioteca necessária
from xgboost import XGBRegressor

# Treinar o modelo XGBoost
xgboost_model = XGBRegressor(n_estimators=100, random_state=42)
xgboost_model.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred_xgboost = xgboost_model.predict(X_test)

In [70]:
# Avaliando o modelo
xgboost_mse = mean_squared_error(y_test, y_pred_xgboost)
print("Mean Squared Error:", np.sqrt(xgboost_mse))

Mean Squared Error: 0.783226904182991


**Comentário**:

Ao analisar e comparar os modelos com base em suas métricas de Erro Quadrático Médio (EQM), observa-se que o modelo de Random Forest apresentou desempenho.

O próximo passo, é avaliar a capacidade de generalização do modelo. Para isso, será utilizada a validação cruzada.


A validação cruzada desempenha um papel fundamental na estimativa da capacidade do modelo em generalizar para dados não observados. Esse procedimento oferece uma avaliação mais robusta e confiável de sua performance, assegurando que o modelo consiga realizar previsões consistentes em diferentes subconjuntos do conjunto de dados.

### Cross validation

In [73]:
# Importando a biblioteca necessária
from sklearn.model_selection import cross_val_score

In [74]:
# Ridge Regressão

# Calculando os scores de validação cruzada usando a Regressão Linear
scores = cross_val_score(ridge, X_train, y_train,
                         scoring="neg_mean_squared_error", cv=10)

# Calculando a raiz quadrada negativa do erro quadrático médio para cada fold
ridge_rmse_scores = np.sqrt(-scores)

# Definindo uma função para exibir os scores
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

# Exibindo os scores de validação cruzada
display_scores(ridge_rmse_scores)

Scores: [0.80033647 0.79293305 0.67244363 0.84029676 0.82887427 0.76658917
 0.6804378  0.76796787 0.79396689 0.74979485]
Mean: 0.769364076700138
Standard deviation: 0.053279501037369704


In [75]:
# Lasso Regressão

# Calculando os scores de validação cruzada usando a Regressão Linear
scores = cross_val_score(lasso, X_train, y_train,
                         scoring="neg_mean_squared_error", cv=10)

# Calculando a raiz quadrada negativa do erro quadrático médio para cada fold
lasso_rmse_scores = np.sqrt(-scores)

# Definindo uma função para exibir os scores
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

# Exibindo os scores de validação cruzada
display_scores(lasso_rmse_scores)

Scores: [0.79614104 0.75800927 0.67502657 0.85148871 0.84147582 0.7329132
 0.68232457 0.75381349 0.77521934 0.72147155]
Mean: 0.7587883555878
Standard deviation: 0.05660913574199742


In [76]:
# ElasticNet

# Calculando os scores de validação cruzada usando a Regressão Linear
scores = cross_val_score(elastic, X_train, y_train,
                         scoring="neg_mean_squared_error", cv=10)

# Calculando a raiz quadrada negativa do erro quadrático médio para cada fold
elastic_rmse_scores = np.sqrt(-scores)

# Definindo uma função para exibir os scores
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

# Exibindo os scores de validação cruzada
display_scores(elastic_rmse_scores)

Scores: [0.79458497 0.76219159 0.66876586 0.84970879 0.82722194 0.73317209
 0.68197302 0.75598904 0.76550974 0.72662519]
Mean: 0.7565742228933101
Standard deviation: 0.05471924416695534


In [77]:
# Random Forest

# Calculando os scores de validação cruzada usando a Floresta Aleatória
scores = cross_val_score(random_forest, X_train, y_train,
                         scoring="neg_mean_squared_error", cv=10)

# Calculando a raiz quadrada negativa do erro quadrático médio para cada fold
forest_rmse_scores = np.sqrt(-scores)

# Exibindo os scores de validação cruzada
display_scores(forest_rmse_scores)

Scores: [0.66588653 0.7315617  0.73011403 0.72305362 0.73753883 0.772255
 0.62783832 0.68317314 0.76456862 0.75103002]
Mean: 0.7187019818147937
Standard deviation: 0.0435822486754633


In [78]:
# Decision Tree

# Calculando os scores de validação cruzada usando a árvore de decisão
scores = cross_val_score(decision_tree, X_train, y_train,
                         scoring="neg_mean_squared_error", cv=10)

# Calculando a raiz quadrada negativa do erro quadrático médio para cada fold
tree_rmse_scores = np.sqrt(-scores)

# Exibindo os scores de validação cruzada
display_scores(tree_rmse_scores)

Scores: [0.88054301 1.0757545  0.99954517 1.23047381 1.08379263 0.98692292
 0.98985003 1.0531045  0.97314665 0.97114827]
Mean: 1.0244281473859425
Standard deviation: 0.08878547485813003


In [79]:
# XGBoost

# Criando o modelo XGBoost
xgboost_model = XGBRegressor(n_estimators=100, random_state=42)

# Calculando os scores de validação cruzada usando XGBoost
scores = cross_val_score(xgboost_model, X_train, y_train,
                         scoring="neg_mean_squared_error", cv=10)

# Calculando a raiz quadrada negativa do erro quadrático médio para cada fold
xgboost_rmse_scores = np.sqrt(-scores)

# Exibindo os scores de validação cruzada
display_scores(xgboost_rmse_scores)

Scores: [0.66977791 0.74102202 0.76256321 0.7949095  0.76441041 0.769869
 0.6721842  0.71810947 0.8232937  0.80370403]
Mean: 0.7519843465355798
Standard deviation: 0.049580582049580574


In [80]:
# Criando o dataframe para melhor visualizar
df_scores = pd.DataFrame({
    'Modelo': ['Ridge Regression', 'Lasso Regression', "ElasticNet", 'Random Forest', 'Decision Tree', 'XGBoost'],
    'Scores': [ridge_rmse_scores.mean(), lasso_rmse_scores.mean(), elastic_rmse_scores.mean(),
               forest_rmse_scores.mean(), tree_rmse_scores.mean(), xgboost_rmse_scores.mean()],
    'Desvio Padrão': [ridge_rmse_scores.std(), lasso_rmse_scores.std(), elastic_rmse_scores.std(),
                      forest_rmse_scores.std(), tree_rmse_scores.std(), xgboost_rmse_scores.std()]
})

In [81]:
# Visualizando o dataframe
df_scores.round(2)

,Modelo,Scores,Desvio Padrão
0,Ridge Regression,0.77,0.05
1,Lasso Regression,0.76,0.06
2,ElasticNet,0.76,0.05
3,Random Forest,0.72,0.04
4,Decision Tree,1.02,0.09
5,XGBoost,0.75,0.05


O modelo Ridge Regression se destacou por sua capacidade de generalização, apresentando uma média de 0,77 e desvio padrão de 0,05 nos scores de validação cruzada.

### Otimização de hiperparâmetros

In [82]:
# importando a bibloteca Grid Search
from sklearn.model_selection import GridSearchCV

In [83]:
# definindo o Grid de valores possíveis de alpha
param_grid = {'alpha': [0.001, 0.01, 0.1, 1, 10, 100, 200]}

# Aplicando o GridSearch com validação cruzada
grid = GridSearchCV(ridge, param_grid, cv=5, scoring='neg_mean_squared_error')

# Treinando o modelo
grid.fit(X_train, y_train)

# Imprimindo os melhores alphas do modelo
print("Melhor alpha:", grid.best_params_['alpha'])
print("Melhor score:", -grid.best_score_)

Melhor alpha: 10
Melhor score: 0.5849996847787893


In [84]:
# isntanciando o modelo e alterando o apha
ridge = Ridge(alpha=10)

# ajuste do modelo com os dados de teste
ridge.fit(X_train, y_train)

Ridge(alpha=10)

In [85]:
# realizando a predição com o modelo reajustado
y_pred_ridge = ridge.predict(X_test)

In [86]:
# Calculando os scores de validação cruzada usando a Regressão Linear
scores = cross_val_score(ridge, X_train, y_train,
                         scoring="neg_mean_squared_error", cv=10)

# Calculando a raiz quadrada negativa do erro quadrático médio para cada fold
ridge_rmse_scores = np.sqrt(-scores)

# Exibindo os scores de validação cruzada
display_scores(ridge_rmse_scores)

Scores: [0.79631264 0.7804278  0.66707876 0.85220951 0.82696553 0.7505672
 0.68260674 0.75834654 0.7764574  0.73255912]
Mean: 0.7623531229175354
Standard deviation: 0.05514267058024877


## Salvando o modelo e prevendo o dado

In [87]:
# Salvando o modelo

# importando a biblioteca
import pickle

# Salvando o modelo treinado
with open('modelo_ridge_regression.pkl', 'wb') as file:
    pickle.dump(ridge, file)

In [88]:
# prevendo com o dado fornecido no desafio

# Criando o dicionário com os dados que pretende prever
linha = {
    'Released_Year': 1994,
    'A': 1,
    'Drama': 1,
    'Runtime_clean': 142,
    'Meta_score': 80.0,
    'No_of_Votes': 2343110,
    'Gross': 28341469,
}

# Transformando em DataFrame com as colunas do X_train
df_previsao = pd.DataFrame([linha], columns=X_train.columns)

# preenchendo 0 onde não há valor
df_previsao = df_previsao.fillna(0)

In [89]:
# Criando uma coluna e atribuindo 0
df_previsao['IMDB_Rating'] = 0

# Normalizando as colunas numéricas
df_previsao[['Runtime_clean', 'IMDB_Rating', 'Meta_score', 'No_of_Votes', 'Gross']] = scaler.transform(
    df_previsao[['Runtime_clean', 'IMDB_Rating', 'Meta_score', 'No_of_Votes', 'Gross']]
)

# Eliminando a coluna "IMDB_Rating" que só serviu para normalizar
df_previsao = df_previsao.drop('IMDB_Rating', axis=1)

In [90]:
# Visualizando o datraframe para prever
df_previsao

,Released_Year,Meta_score,No_of_Votes,Gross,Runtime_clean,Action,Adventure,Animation,Biography,Comedy,...,Cert_Passed,Cert_R,Cert_TV-14,Cert_TV-MA,Cert_TV-PG,Cert_Thriller,Cert_U,Cert_U/A,Cert_UA,Cert_Unrated
0,1994,0.152113,6.458226,-0.363747,0.681028,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [91]:
# realizando a predição com o dado fornecido
y_pred = ridge.predict(df_previsao)

# Imprimindo em tela o resultado
print(f"A nota IMDB prevista para o filme The Shawshank Redemption é: {y_pred[0].round(2)}")

A nota IMDB prevista para o filme The Shawshank Redemption é: 4.91


## Modelo de Processamento de Linguagem Natural

### Insights obtidos na coluna Overwiew

In [92]:
# Importando as bibliotecas
import re
from wordcloud import WordCloud
import plotly.graph_objects as go
from PIL import Image

Filtraremos o dataframe pelo gênero "Drama", pois é o gênero aparece com maior frequência no dataset.

In [93]:
# Cpoiando o dataframe
df_pln_cloud = df_copiado.copy()

In [94]:
# Cria-se uma função para fazer o préprocessamento
def processa_texto(text):
    text = text.lower() # converte o texto para minúsculo
    text = re.sub(r'[^\w\s]', '', text) # remove pontuação
    return text

In [95]:
# Aplicando a limpeza na coluna Overview
df_pln_cloud['Clean_Overview'] = df_pln_cloud['Overview'].apply(processa_texto)

In [96]:
# Gerando o texto para o gênero Drama
texto = " ".join(df_pln_cloud[df_pln_cloud['Genre'] == 'Drama']['Clean_Overview'])

In [97]:
# Gerando a nuvem de palavras
wordcloud = WordCloud(stopwords=None, background_color='black', colormap='Reds').generate(texto)

# Convertendo o objeto wordcloud para uma imagem
wordcloud_image = wordcloud.to_image()

# Criando uma figura
fig = go.Figure()

# Definindo o layout da imagem
fig.add_layout_image(
    dict(
        source=wordcloud_image,
        xref="x",
        yref="y",
        x=0,
        y=1,
        sizex=1,
        sizey=1,
        sizing="stretch",
        layer="below"
    )
)

# o layout do gráfico
fig.update_layout(
    title_text="Nuvem de Palavras do Gênero Drama",
    xaxis_visible=False,
    yaxis_visible=False,
    xaxis_range=[0, 1],
    yaxis_range=[0, 1],
    margin=dict(l=0, r=0, t=30, b=0)
)

# Imprimindo em tela
fig.show()

Vamos repetir para o segundo colocado "Drama, Romance" e "Comedia, Drama"

In [98]:
# Gerando o texto para o gênero Drama
texto = " ".join(df_pln_cloud[df_pln_cloud['Genre'] == 'Drama, Romance']['Clean_Overview'])

In [99]:
# Gerando a nuvem de palavras
wordcloud = WordCloud(stopwords=None, background_color='black', colormap='cividis').generate(texto)

# Convertendo o objeto wordcloud para uma imagem
wordcloud_image = wordcloud.to_image()

# Criando uma figura
fig = go.Figure()

# Definindo o layout da imagem
fig.add_layout_image(
    dict(
        source=wordcloud_image,
        xref="x",
        yref="y",
        x=0,
        y=1,
        sizex=1,
        sizey=1,
        sizing="stretch",
        layer="below"
    )
)

# o layout do gráfico
fig.update_layout(
    title_text="Nuvem de Palavras do Gênero Drama, Romance",
    xaxis_visible=False,
    yaxis_visible=False,
    xaxis_range=[0, 1],
    yaxis_range=[0, 1],
    margin=dict(l=0, r=0, t=30, b=0)
)

# Imprimindo em tela
fig.show()

In [100]:
# Gerando o texto para o gênero Drama
texto = " ".join(df_pln_cloud[df_pln_cloud['Genre'] == 'Comedy, Drama']['Clean_Overview'])

In [101]:
# Gerando a nuvem de palavras
wordcloud = WordCloud(stopwords=None, background_color='black', colormap='viridis').generate(texto)

# Convertendo o objeto wordcloud para uma imagem
wordcloud_image = wordcloud.to_image()

# Criando uma figura
fig = go.Figure()

# Definindo o layout da imagem
fig.add_layout_image(
    dict(
        source=wordcloud_image,
        xref="x",
        yref="y",
        x=0,
        y=1,
        sizex=1,
        sizey=1,
        sizing="stretch",
        layer="below"
    )
)

# o layout do gráfico
fig.update_layout(
    title_text="Nuvem de Palavras do Gênero Comédia, Drama",
    xaxis_visible=False,
    yaxis_visible=False,
    xaxis_range=[0, 1],
    yaxis_range=[0, 1],
    margin=dict(l=0, r=0, t=30, b=0)
)

# Imprimindo em tela
fig.show()

### Inferência

In [ ]:
# Importando as bibliotecas
from sklearn.utils import resample
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
# Visualizando a coluna de Genre e contar a quantidade de vezes que aparece os gêneros
df_copiado['Genre'].value_counts()

,count
Genre,
Drama,84
"Drama, Romance",37
"Comedy, Drama",35
"Comedy, Drama, Romance",31
"Action, Crime, Drama",30
...,...
"Action, Adventure, Family",1
"Action, Crime, Mystery",1
"Animation, Drama, Romance",1


Observa-se que o gênero 'Drama' é predominante neste conjunto de dados. Se um modelo for treinado com essa base, ele pode se tornar enviesado e tender a classificar a maioria dos dados como 'Drama', ignorando as outras categorias. Para resolver isso, aplicaremos a técnica de oversampling. Este é um método de rebalanceamento que aumenta o número de amostras dos gêneros minoritários, duplicando-as para que todas as categorias tenham uma representatividade mais equilibrada.

In [ ]:
# Separando os dados em Drama e não Drama
df_drama = df_copiado[df_copiado['Genre'] == 'Drama']
df_outros = df_copiado[df_copiado['Genre'] != 'Drama']

In [ ]:
# Obtendo as classes minoritárias
classes_minoritarias = df_outros['Genre'].unique()

# Cria uma lista vazia para armazenar
oversample = []

# Cria o laço for para aplicar oversampling
for genre in classes_minoritarias:
    df_genero_s = df_outros[df_outros['Genre'] == genre]
    df_upsampled = resample(df_genero_s,
                            replace=True,  # permite repetir
                            n_samples=len(df_drama),  # igual ao Drama
                            random_state=42)
    oversample.append(df_upsampled)

# Convertendo em dataframe
df_balanceada = pd.concat([df_drama] + oversample)

In [ ]:
# Visualizando o dataframe balanceado
print(df_balanceada.shape)

(16968, 15)


In [ ]:
# Definindo as variáveis que vão ser inferidas
X = df_balanceada["Overview"]
y = df_balanceada["Genre"]

In [ ]:
# Separando os dados de traino e teste (com 70% de treino e 30% de teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# Criando o pipeline, o objetivo de criar o pipeline é vetorizar o texto e treinar o classificador em sequência
model_pipeline = make_pipeline(
    TfidfVectorizer(stop_words='english'), # Converte texto em vetores numéricos, removendo stopwords em inglês
    MultinomialNB() # instanciandio o classificador Naive Bayes
)

In [ ]:
# Treinando o modelo
model_pipeline.fit(X_train, y_train)

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer(stop_words='english')),
                ('multinomialnb', MultinomialNB())])

In [ ]:
# Prevendo com os dados de teste
y_pred = model_pipeline.predict(X_test)

In [ ]:
# Imprimindo os resultados
print(f"Dados de Teste: {y_test.values}")
print(f"Previsões do Modelo: {y_pred}")

Dados de Teste: ['Action, Comedy, Crime' 'Crime, Film-Noir, Mystery'
 'Crime, Drama, Musical' ... 'Drama, Fantasy, War'
 'Drama, History, Romance' 'Adventure, Horror, Sci-Fi']
Previsões do Modelo: ['Action, Comedy, Crime' 'Crime, Film-Noir, Mystery'
 'Crime, Drama, Musical' ... 'Drama, Fantasy, War'
 'Drama, History, Romance' 'Adventure, Horror, Sci-Fi']


In [ ]:
# Obtendo acurácia
accuracy = accuracy_score(y_test, y_pred)

# Imprimindo acurácia
print(f"\nAcurácia do Modelo: {accuracy:.2f}")


Acurácia do Modelo: 0.98


In [ ]:
# Imprimindo o o texto de relatório classificação
print("Relatório de classificação:")

# imprimindo o relatório detalhado
print(classification_report(y_test, y_pred, zero_division=0)) # zero_division = 0 evita avisos caso uma classe não tenha previsões

Relatório de classificação:
                               precision    recall  f1-score   support

            Action, Adventure       1.00      1.00      1.00        33
 Action, Adventure, Biography       1.00      1.00      1.00        29
    Action, Adventure, Comedy       1.00      1.00      1.00        27
     Action, Adventure, Crime       0.90      1.00      0.95        27
     Action, Adventure, Drama       1.00      1.00      1.00        31
    Action, Adventure, Family       1.00      1.00      1.00        28
   Action, Adventure, Fantasy       1.00      1.00      1.00        21
   Action, Adventure, History       0.92      1.00      0.96        22
    Action, Adventure, Horror       1.00      1.00      1.00        24
   Action, Adventure, Mystery       1.00      1.00      1.00        30
   Action, Adventure, Romance       1.00      1.00      1.00        29
    Action, Adventure, Sci-Fi       1.00      0.96      0.98        27
  Action, Adventure, Thriller       1.00      1.

In [ ]:
# Criando a matriz de confusão
matriz_confusao = confusion_matrix(y_test, y_pred, labels=y_test.unique())

# Transformando no dataframe
df_confusao = pd.DataFrame(matriz_confusao, index=y_test.unique(), columns=y_test.unique())

In [ ]:
# Criando uma figura Imshow
fig = px.imshow(df_confusao,
                text_auto=True,
                color_continuous_scale='Blues',
                labels=dict(x="Predito", y="Real", color="Contagem"),
                x=df_confusao.columns,
                y=df_confusao.index)

# Configurando o layout
fig.update_layout(
    title='Matriz de Confusão',
    xaxis_title='Predito',
    yaxis_title='Real',
    width=1000,   # largura em pixels
    height=800    # altura em pixels
)

# imprimindo a figura
fig.show()

## Modelo de Recomendação de Filme

Para recomendar filmes para uma nova pessoa, que ainda não possuem um histórico de perfil, adotaremos uma estratégia baseada em popularidade e qualidade. Os critérios de seleção são:

- Filmes com avaliação no IMDb igual ou superior a 7.5 (alta qualidade).

- Filmes com um número elevado de votos, o que indica que são amplamente conhecidos e assistidos (alta popularidade).

In [ ]:
# Filtrar filmes que possuem boas avaliações e popularidade
excelente_filmes = df_copiado[(df_copiado['IMDB_Rating'] >= 7.5) & (df_copiado['No_of_Votes'] >= 5000)]

# Ordenando por nota IMDB e número de votos
excelente_filme = excelente_filmes.sort_values(by=['IMDB_Rating', 'No_of_Votes'], ascending=False)

# Selecionaando os 10 filmes
top_recomendacao = excelente_filme.head(10)

In [ ]:
# Cria uma lista vazia para armazenar
recomendacao = []

# Criar um loop e inserir justificativa
for _, row in top_recomendacao.iterrows():
    rec = f"Filme: {row['Series_Title']} | Gênero: {row['Genre']} | Nota IMDB: {row['IMDB_Rating']:.1f} | Votos: {int(row['No_of_Votes'])}.\n"
    recomendacao.append(rec)

# Imprimir as recomendações
for r in recomendacao:
    print(r)
    print('-'*80)

Filme: The Godfather | Gênero: Crime, Drama | Nota IMDB: 9.2 | Votos: 1620367.

--------------------------------------------------------------------------------
Filme: The Dark Knight | Gênero: Action, Crime, Drama | Nota IMDB: 9.0 | Votos: 2303232.

--------------------------------------------------------------------------------
Filme: The Godfather: Part II | Gênero: Crime, Drama | Nota IMDB: 9.0 | Votos: 1129952.

--------------------------------------------------------------------------------
Filme: 12 Angry Men | Gênero: Crime, Drama | Nota IMDB: 9.0 | Votos: 689845.

--------------------------------------------------------------------------------
Filme: Pulp Fiction | Gênero: Crime, Drama | Nota IMDB: 8.9 | Votos: 1826188.

--------------------------------------------------------------------------------
Filme: The Lord of the Rings: The Return of the King | Gênero: Action, Adventure, Drama | Nota IMDB: 8.9 | Votos: 1642758.

-------------------------------------------------------

## Requirements

In [ ]:
# Importando a biblioteca
import subprocess

In [ ]:
# Criando o nome do arquivo
arquivo = 'requirements.txt'

In [ ]:
# Criando o arquivo e salvando
with open(arquivo, 'w') as f:
    subprocess.check_call(['pip', 'freeze'], stdout=f)

# Imprimindo a mensagem
print(f"Arquivo '{arquivo}' criado com sucesso!")

Arquivo 'requirements.txt' criado com sucesso!
